In [1]:
from tqdm import tqdm
raw_train_sentences = []
for i in tqdm(range(10000)):
    five_sentences = ""
    for line in open("./all/data/descriptions_train/" + str(i) + ".txt"):
        five_sentences += line.strip() + " "
    raw_train_sentences.append(five_sentences.strip())

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1349.20it/s]


In [3]:
print(len(raw_train_sentences))
print(raw_train_sentences[:1])

10000
['The skateboarder is putting on a show using the picnic table as his stage. A skateboarder pulling tricks on top of a picnic table. A man riding on a skateboard on top of a table. A skate boarder doing a trick on a picnic table. A person is riding a skateboard on a picnic table with a crowd watching.']


In [4]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [5]:
import numpy as np

embeddings_index = {}
f = open('glove.42B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

1917495it [07:14, 4410.48it/s]


In [7]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

xtrain_glove = [sent2vec(x) for x in tqdm(raw_train_sentences)]


 22%|█████████████████                                                           | 2250/10000 [00:07<00:14, 546.20it/s]


 63%|███████████████████████████████████████████████▋                            | 6282/10000 [00:14<00:06, 562.04it/s]


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:21<00:00, 465.73it/s]

In [12]:
print(len(xtrain_glove[0]))
print(xtrain_glove[0])

300
[-6.59739673e-02  6.31461572e-03  1.76676270e-02 -2.09363811e-02
  6.97108209e-02 -5.53418957e-02 -4.78366733e-01 -2.87478119e-02
 -2.87563410e-02 -3.81933264e-02  8.70583020e-03 -6.88630864e-02
 -2.51761843e-02 -2.95907669e-02  5.55269271e-02  9.40703694e-03
 -8.62565264e-03 -6.24382542e-03  2.63830135e-03 -5.10619394e-03
  1.83799062e-02 -1.51503310e-02 -2.48604156e-02  6.11835858e-03
 -1.88628398e-02 -4.62049320e-02  2.83406749e-02  8.64228699e-03
 -9.40761622e-03  4.78029922e-02  8.68050754e-03  3.37866284e-02
 -4.17832211e-02 -6.24777284e-03 -8.87036789e-03 -1.34112302e-03
 -6.93938881e-02  3.65715548e-02 -8.57924670e-03  4.99792136e-02
  1.18220160e-02  1.30151808e-02  2.26914417e-02 -4.29303944e-03
 -5.33251092e-03  1.38035454e-02 -7.25924456e-03 -1.52796637e-02
 -1.58814061e-02 -2.79267170e-02 -1.31732933e-02 -7.52149243e-03
 -4.24099751e-02  1.82855073e-02  9.70232766e-03 -2.10205875e-02
 -7.02355877e-02  4.26505413e-03 -1.87394326e-03  3.45298983e-02
 -1.24890823e-03 -1.8

In [14]:
import csv

In [19]:
np.savetxt("query_glove.csv", xtrain_glove, delimiter=",")

In [21]:
# Reading 1000-d train features
features_train = np.zeros((10000,1000))
with open('./all/data/features_train/features_resnet1000_train.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[13:]
        row = []
        for i in range(1,len(line)):
            row.append(float(line[i]))
        features_train[int(image_name)] = row

In [25]:
print(len(features_train))
print(features_train[0])

10000
[-4.55291319e+00 -3.33937263e+00  7.39804506e-01 -1.06496036e+00
 -3.85820955e-01 -1.37479579e+00 -3.17297888e+00 -1.21654260e+00
 -4.12231970e+00 -4.73385930e-01 -1.79821754e+00 -2.46816111e+00
 -2.25911903e+00 -2.29419303e+00 -1.74317026e+00 -2.02084851e+00
 -2.27003956e+00 -2.49910641e+00 -1.36317313e+00 -2.79533458e+00
 -2.93762565e+00 -1.39555442e+00 -1.84041190e+00 -2.10604668e+00
 -4.01298809e+00 -2.55336666e+00 -2.64755726e+00 -3.65012050e+00
 -3.83733988e+00 -1.63596416e+00 -4.35165787e+00 -2.90704250e+00
 -3.37037444e+00 -3.25298595e+00  1.32412702e-01 -3.18060565e+00
 -1.76535618e+00 -5.72201633e+00 -7.55883217e-01  2.20053360e-01
 -2.53787899e+00 -4.96546656e-01  7.50343382e-01  1.66791797e-01
 -1.27497387e+00 -6.29044950e-01 -1.90589905e+00  1.27659619e+00
 -5.12576222e-01 -2.52963758e+00 -7.91163981e-01  2.01129222e+00
 -1.20844972e+00 -2.97828269e+00 -1.46261716e+00 -3.48112607e+00
 -4.04569149e+00 -4.54287195e+00 -3.18323541e+00  8.17079842e-01
 -7.50888526e-01 -3

 -2.18719125e+00 -4.32532072e+00 -1.36100817e+00 -1.84894502e-01]


In [28]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100, verbose=3)
regr.fit(xtrain_glove, features_train)

building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.7s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   43.7s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 30.8min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=3, warm_start=False)

In [29]:
raw_test_sentences = []
for i in tqdm(range(2000)):
    five_sentences = ""
    for line in open("./all/data/descriptions_test/" + str(i) + ".txt"):
        five_sentences += line.strip() + " "
    raw_test_sentences.append(five_sentences.strip())


100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:06<00:00, 303.57it/s]

In [30]:
xtest_glove = [sent2vec(x) for x in tqdm(raw_test_sentences)]


 37%|█████████████████████████████                                                 | 745/2000 [00:07<00:10, 122.27it/s]


 83%|███████████████████████████████████████████████████████████████▋             | 1655/2000 [00:14<00:02, 126.84it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:17<00:00, 112.37it/s]

In [31]:
regression_test = regr.predict(xtest_glove)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.4s finished


In [32]:
print(regression_test.shape)

(2000, 1000)


In [34]:
# for i in range(2000):
#     guess = regression_test[i]
#     for y in range(len())

# Reading 1000-d test features
features_test = np.zeros((2000,1000))
with open('./all/data/features_test/features_resnet1000_test.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for line in csv_reader:
        image_name = line[0].strip(".jpg")[12:]
        row = []
        for i in range(1,len(line)):
            row.append(float(line[i]))
        features_test[int(image_name)] = row

In [36]:
rank_list = []
for i in range(2000):
    guess = regression_test[i]
    dist = []
    for y in range(2000):
        dist.append((np.linalg.norm(guess-features_test[y]), y))
    dist.sort()
    rank_list.append([v for (k,v) in dist[:20]])

In [41]:
print(rank_list[0])

[68, 912, 46, 849, 305, 1243, 808, 216, 1088, 1018, 87, 1882, 673, 1231, 1384, 153, 1850, 629, 785, 1913]


In [44]:
def mergeTwenty(lst):
    result = str(lst[0])+".jpg"
    for i in range(1,len(lst)):
        result += " " + str(lst[i]) + ".jpg"
    return result

with open('1k_submission.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(["Descritpion_ID", "Top_20_Image_IDs"])
    for i in range(len(rank_list)):
        csv_writer.writerow([str(i) + ".txt", mergeTwenty(rank_list[i])])